<a href="https://colab.research.google.com/github/Emerson-Foco/Chat_Service_Evaluation_AI_Gemini_API/blob/main/Chat_Service_Evaluation_AI_Gemini_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# First Steps

- [ ] Install and import Google SDK

In [10]:
!pip install -q -U google-generativeai

- [ ] Import Goodle SDK

In [11]:
import pathlib
import textwrap

import google.generativeai as genai

from IPython.display import display
from IPython.display import Markdown


def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

- [ ] Setting your API key by *userdata.get*

In [12]:
from google.colab import userdata
api_key = userdata.get('SECRET_KEY')

genai.configure(api_key=api_key)

# Setting your employee

> Adicionar aspas



- [ ] employee info

In [ ]:
employee_name = 'Raquel'
employee_position = 'atendente'
employee_company = 'Google'
employee_products = 'AI Gemini'

employee_info = {f'Você se chama {employee_name} e é {employee_position} de chat e trabalha para empresa {employee_company} '
                  f'com os produtos {employee_products}, realize um atendimento natural, humanizado e '
                  'sem assustar o cliente. Você não pode mentir e nem inventar.'
                  ' O que você não souber, diga que não possui a informação'}

print(employee_info)

- [ ] employee model settings

In [31]:
employee_generation_config = {
    'candidate_count': 1,
    'temperature': 0.9,
}

employee_safety_settings = {
    'HARASSMENT': 'BLOCK_NONE',
    'HATE': 'BLOCK_NONE',
    'SEXUAL': 'BLOCK_NONE',
    'DANGEROUS': 'BLOCK_NONE',
}

employee_model = genai.GenerativeModel('gemini-pro', generation_config=employee_generation_config, safety_settings=employee_safety_settings)

#Setting customer

- [ ] customer info

In [ ]:
customer_name = 'Paulo'
customer_position = 'CEO'
customer_company = 'Alura'
customer_interest = 'IA para ajudar os alunos da sua plataforma de cursos online'

customer_info = {f'Você é {customer_name}, {customer_position} da empresa {customer_company} e'
                  f' Você tem interesse em {customer_interest}.'}

print(customer_info)

- [ ] customer model settings

In [20]:
customer_generation_config = {
    'candidate_count': 1,
    'temperature': 0.9,
}

customer_safety_settings = {
    'HARASSMENT': 'BLOCK_NONE',
    'HATE': 'BLOCK_NONE',
    'SEXUAL': 'BLOCK_NONE',
    'DANGEROUS': 'BLOCK_NONE',
}

customer_model = genai.GenerativeModel('gemini-pro', generation_config=customer_generation_config, safety_settings=customer_safety_settings)

# Chat: AI vs AI

In [32]:
employee_chat = employee_model.start_chat(history=[])
customer_chat = customer_model.start_chat(history=[])

In [ ]:
employee_chat.history = []
customer_chat.history = []

prompt = ''

employee = ''
customer = customer_chat.send_message(f'{customer_info}\nInicie a conversa com o/a atendente do chat')

while prompt != 'stop':
  print('customer: ', customer.text, '\n')
  employee = employee_chat.send_message(f'{employee_info} {customer.text}')
  print('employee: ', employee.text, '\n')
  customer = customer_chat.send_message(f'{customer_info} {employee.text}')

  prompt = input('digite "stop" para parar ou enter para continuar: ')